In [1]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup
from selenium import webdriver
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests
import os
import pprint as pp
import time

In [2]:
# INITIALIZE CHROMEDRIVER AS BROWSWER
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

browser = init_browser()

### VISIT MARS.NASA.GOV FOR MARS NEWS

In [3]:

url = 'https://mars.nasa.gov/news/'
browser.visit(url)
time.sleep(5)

In [4]:
#PARSE HMTL FROM BROWSER AND FIND THE NEWS
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
all_news = soup.find_all('div', class_='list_text',limit=1)

In [5]:
latest_news = all_news[0]
latest_news

<div class="list_text"><div class="list_date">May 21, 2020</div><div class="content_title"><a href="/news/8677/air-deliveries-bring-nasas-perseverance-mars-rover-closer-to-launch/" target="_self">Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch</a></div><div class="article_teaser_body">A NASA Wallops Flight Facility cargo plane transported more than two tons of equipment — including the rover's sample collection tubes — to Florida for this summer's liftoff.</div></div>

In [6]:
latest_title = latest_news.find('div', class_='content_title').text
latest_title

"Air Deliveries Bring NASA's Perseverance Mars Rover Closer to Launch"

In [7]:
latest_para = latest_news.find('div', class_='article_teaser_body').text
latest_para

"A NASA Wallops Flight Facility cargo plane transported more than two tons of equipment — including the rover's sample collection tubes — to Florida for this summer's liftoff."

### Find Image of Mars

In [8]:
#Find image of Mars
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(5)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [21]:
# Use the image ID to click link to full_image
browser.click_link_by_id('full_image')

In [22]:
# Use partial href to click for full image page
browser.click_link_by_partial_href('spaceimages/details.php')

In [23]:
#Scrape the full html from the page
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [24]:
# Find the link to the main_image class for the .jpg file
main_image = soup.find('img', class_='main_image')
main_image['src']

'/spaceimages/images/largesize/PIA17761_hires.jpg'

In [25]:
#Get the full URL for the main image
main_image = f"https://www.jpl.nasa.gov/{main_image['src']}"
print(main_image)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA17761_hires.jpg


### Find the weather on Mars with Twitter

#### Citation: I followed a guide from below link to scrape tweets using beautifulsoup

#### https://notes.ayushsharma.in/2018/09/a-guide-to-web-scraping-in-python-using-beautifulsoup

In [26]:
url = 'https://twitter.com/marswxreport?lang=en'

In [85]:
data = requests.get(url)

In [86]:
html = BeautifulSoup(data.text, 'html.parser')
tweets = html.select('#timeline li.stream-item')

In [90]:
for tweet in tweets:
    tweet_id = tweet['data-item-id']
    tweet_text = tweet.select('p.tweet-text')[0].get_text()

In [91]:
print(f"Tweet ID: {tweet_id} \nTweet Text: {tweet_text}")

Tweet ID: 1258137257271341056 
Tweet Text: InSight sol 512 (2020-05-05) low -92.9ºC (-135.2ºF) high -3.1ºC (26.4ºF)
winds from the SW at 4.8 m/s (10.7 mph) gusting to 19.2 m/s (42.9 mph)
pressure at 6.80 hPapic.twitter.com/TJRWhGbUMa


### Get Mars facts and put into dataframe

In [92]:
url = 'https://space-facts.com/mars/'
response = requests.get(url)

In [111]:
mars_facts_df = pd.read_html(response.text)

In [112]:
mars_facts_df = mars_facts_df[0]
mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [230]:
url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


In [277]:
response = requests.get(url)
time.sleep(1)
soup = BeautifulSoup(response.text, 'html.parser')

In [234]:
#Elements 4-7 contain the 4 URLs I want to visit
links = soup.find_all('a',href=True)
links_index = [4,5,6,7]

In [289]:
photo_links = []
for link in links_index:
    try:
        url = f"https://astrogeology.usgs.gov{links[link]['href']}"
        response = requests.get(url)
        print(url)
        name = url.split("/")[7]
        name = name.split("_")[0]
        print(name)
        print(response)
        time.sleep(1)
        soup = BeautifulSoup(response.text,"html.parser")
        downloads = soup.find('div',class_="downloads")
        original = downloads.find_all('li')[1]
        original = original.find('a')['href']
        photo_links.append({"title": f"{name} hemisphere", "img_url": original})
    except:
        print("error: could not find link")

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
cerberus
<Response [200]>
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
schiaparelli
<Response [200]>
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
syrtis
<Response [200]>
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced
valles
<Response [200]>


In [290]:
photo_links

[{'title': 'cerberus hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'schiaparelli hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'syrtis hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'valles hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]